In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [ ]:
PATH='data/amazon_fine_food/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'
os.makedirs(TRN, exist_ok=True)
os.makedirs(VAL, exist_ok=True)
%ls {TRN}

In [ ]:
!ls {PATH}

In [ ]:
TEXT = data.Field(lower=True, tokenize=spacy_tok) # word-level

In [ ]:
TEXT = data.Field(lower=True, tokenize=list) # character-level: list('abc') = ['a', 'b', 'c']

In [ ]:
bs=32; bptt=70

In [ ]:
df = pd.read_csv(f'{PATH}Reviews.csv')

In [ ]:
df.head()

In [ ]:
n = len(df)

In [ ]:
idxs = get_cv_idxs(n)
val_df = df.iloc[idxs].set_index("Id")
samp_size = len(val_df); samp_size

In [ ]:
df.to_csv(f'{PATH}train/train_amazon_fine_food_reviews.csv')
val_df.to_csv(f'{PATH}test/test_amazon_fine_food_reviews.csv')

In [ ]:
df = pd.read_csv(f'{PATH}train/train_amazon_fine_food_reviews.csv')
val_df = pd.read_csv(f'{PATH}test/test_amazon_fine_food_reviews.csv')

In [ ]:
md = LanguageModelData.from_dataframes(PATH, TEXT, 'Text', df, val_df, test_df=val_df, bs=bs, bptt=bptt, min_freq=10)

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [ ]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

In [ ]:
TEXT.vocab.itos[:12]

In [ ]:
TEXT.vocab.stoi['a']

In [ ]:
md.trn_ds[0].text[:12]

In [ ]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

In [ ]:
next(iter(md.trn_dl))

In [ ]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [ ]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [ ]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.2, dropout=0.2, wdrop=0.1, dropoute=0.08, dropouth=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save_encoder('dropout_exp_1_adam3_10_enc')

In [ ]:
learner.lr_find()

In [ ]:
learner.sched.plot()

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10, cycle_save_name='adam3_20')

In [ ]:
learner.save_encoder('dropout_exp_2_adam3_10_enc')

In [ ]:
learner.fit(3e-3, 2, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

In [ ]:
learner.save_encoder('dropout_exp_3_adam3_10_enc')

In [ ]:
learner.save('dropout_exp_3_adam3_20_model')

In [ ]:
learner.load('dropout_exp_3_adam3_20_model')

In [ ]:
m=learner.model
ss="""omg my life is """
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

In [ ]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [ ]:
nexts = torch.topk(res[-1], 20)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

In [ ]:
print(ss,"\n")
for i in range(25):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

In [ ]:
m

In [ ]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [ ]:
SENTIMENT_LABEL = data.Field(sequential=False)

In [ ]:
??LanguageModelData.from_dataframes

In [ ]:
??ConcatTextDatasetFromDataFrames.splits

In [ ]:
torchtext.datasets.IMDB.splits

In [ ]:
md.trn_ds.splits

In [ ]:
class Sentiment140Dataset(torchtext.data.TabularDataset):
    @classmethod
    def splits(cls, train_df=None, val_df=None, test_df=None, **kwargs):
        train_data = None if train_df is None else cls(train_df, **kwargs)
        val_data = None if val_df is None else cls(val_df, **kwargs)
        test_data = None if test_df is None else cls(test_df, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [ ]:
ls {PATH}

In [ ]:
ds = Sentiment140Dataset(f'{PATH}Sentiment-Analysis-Dataset.csv', 'csv', [('Sentiment', SENTIMENT_LABEL), ('SentimentText', TEXT)])

In [ ]:
ds.examples